<a href="https://colab.research.google.com/github/kanchanraiii/SecureRag/blob/master/FAISS_Minor_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.9 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()


Saving ecommerce_dataset.jsonl to ecommerce_dataset.jsonl


In [ ]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
docs = []
with open("ecommerce_dataset.jsonl", "r") as f:
    for line in f:
        obj = json.loads(line)

        # Join all key-value pairs into one text string
        text = " | ".join([f"{k}: {v}" for k, v in obj.items()])

        docs.append(text)

print(f"Loaded {len(docs)} documents")
print("Example doc:", docs[0][:300], "...")


Loaded 2000 documents
Example doc: order_id: ORD412526 | customer_name: Yoshita Sahota | email: sibaldaniel@example.org | phone: 03204410434 | shipping_address: 45/47, Menon Nagar, Mangalore 185592 | billing_address: 63/115, Sarin Ganj, Katni 036380 | product_name: Laborum Electronics | category: Beauty & Personal Care | quantity: 1  ...


In [ ]:
embeddings = model.encode(docs)
embeddings = np.array(embeddings).astype("float32")

d = embeddings.shape[1]   # embedding dimension
index = faiss.IndexFlatL2(d)
index.add(embeddings)

print(f"Indexed {len(docs)} documents")


Indexed 2000 documents


In [ ]:
def search(query, k=3):
    q_vec = model.encode([query]).astype("float32")
    distances, indices = index.search(q_vec, k)
    results = [docs[i] for i in indices[0]]
    return results


In [ ]:
query = "Which customer ordered electronics?"
retrieved = search(query, k=2)

print("Query:", query)
print("Retrieved docs:")
for doc in retrieved:
    print("-", doc[:300], "...\n")

Query: Which customer ordered electronics?
Retrieved docs:
- order_id: ORD587112 | customer_name: Zarna Baria | email: jgade@example.net | phone: 7378441556 | shipping_address: H.No. 344, Goda Ganj, Hapur-380552 | billing_address: 17, Mohan SSreet, Rajahmundry 074441 | product_name: Autem Electronics | category: Toys | quantity: 5 | unit_price: 9035.9 | payme ...

- order_id: ORD311170 | customer_name: Ryan Borra | email: tailorneel@example.net | phone: 08443452745 | shipping_address: H.No. 73, Misra Zila, Ambala 953247 | billing_address: 412, Murthy Street, Howrah-923012 | product_name: RepudianUae Electronics | category: Home & Kitchen | quantity: 4 | unit_pr ...

